In [1]:
from models import load_all_models
model_lst = load_all_models()

Loaded detection model vikp/surya_det3 on device cuda with dtype torch.float16
Loaded detection model vikp/surya_layout3 on device cuda with dtype torch.float16
Loaded reading order model vikp/surya_order on device cuda with dtype torch.float16
Loaded recognition model vikp/surya_rec2 on device cuda with dtype torch.float16
Loaded texify model to cuda with torch.float16 dtype
Loaded recognition model vikp/surya_tablerec on device cuda with dtype torch.float16


In [2]:
from convert import convert_single_pdf
fpath = "/home/jenny/Downloads/Archive 2/scm_metals/NCS000050_Permit (Issuance)_20230524.pdf"
fpath = "/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Application_20150605 (Copy).pdf"
fpath = "/home/jenny/Downloads/Archive 2/wilders_grove_yard_waste/WI0500447/WI0500447_Staff Report_20200911.pdf"
full_text, doc_images, out_meta, pages, text_blocks = convert_single_pdf(fpath, model_lst)

running jenny's convert_single_pdf


Recognizing tables: 100%|██████████| 1/1 [00:00<00:00, 13.92it/s]


processing block 0 of page 0. bbox [168.0, 37.0, 444.0, 66.0]. merged_block_len: 0
processing block 1 of page 0. bbox [247.0, 84.0, 361.0, 96.0]. merged_block_len: 1
	 appending 0 block Text of len 13 for page 0 block 1. block.bbox [247.0, 84.0, 361.0, 96.0] bbox: [168.0, 37.0, 444.0, 96.0]
processing block 2 of page 0. bbox [50.0, 98.0, 519.0, 137.0]. merged_block_len: 2
processing block 3 of page 0. bbox [50.0, 145.0, 192.0, 161.0]. merged_block_len: 3
	 appending 1 block Section-header of len 218 for page 0 block 3. block.bbox [50.0, 145.0, 192.0, 161.0] bbox: [50.0, 98.0, 519.0, 161.0]
processing block 4 of page 0. bbox [50.0, 162.0, 431.0, 185.0]. merged_block_len: 4
	 appending 2 block Text of len 22 for page 0 block 4. block.bbox [50.0, 162.0, 431.0, 185.0] bbox: [50.0, 162.0, 431.0, 185.0]
processing block 5 of page 0. bbox [68.0, 192.0, 195.0, 204.0]. merged_block_len: 5
processing block 6 of page 0. bbox [68.0, 212.0, 272.0, 222.0]. merged_block_len: 6
processing block 7 of p

In [5]:
pages[1].blocks

[Block(bbox=[168.0, 37.0, 444.0, 66.0], lines=[Line(bbox=[168.0, 37.0, 444.0, 52.0], spans=[Span(bbox=[222.0, 49.0, 222.0, 49.0], text=' ', span_id='1_1', font='Helvetica_non_symbolic', font_weight=-1.0, font_size=1.0, bold=None, italic=None, image=None), Span(bbox=[301.0, 49.0, 301.0, 49.0], text=' ', span_id='1_3', font='Helvetica_non_symbolic', font_weight=-1.0, font_size=1.0, bold=None, italic=None, image=None), Span(bbox=[350.0, 49.0, 350.0, 49.0], text=' ', span_id='1_5', font='Helvetica_non_symbolic', font_weight=-1.0, font_size=1.0, bold=None, italic=None, image=None), Span(bbox=[411.0, 49.0, 411.0, 49.0], text=' ', span_id='1_7', font='Helvetica_non_symbolic', font_weight=-1.0, font_size=1.0, bold=None, italic=None, image=None), Span(bbox=[414.0, 38.0, 444.0, 52.0], text='FOR', span_id='1_8', font='Helvetica_non_symbolic', font_weight=0.0, font_size=15.0, bold=None, italic=None, image=None), Span(bbox=[444.0, 49.0, 444.0, 49.0], text='', span_id='1_9', font='Helvetica_non_symb

In [3]:
[[block.bbox, block.text, block.id] for block in text_blocks]

[[[168.0, 37.0, 444.0, 96.0], '# For Uic\n', '1|-1'],
 [[50.0, 98.0, 519.0, 161.0],
  'Permit No. WI0500447 Date: 09/11/2020 County: Wake To: Michael Rogers Permittee/ Applicant: City of Raleigh Solid Waste Services Regional Office Reviewer: Brion N. Byers Facility Name: Wilders Grove Landfill ( Closed) ',
  '1|-3'],
 [[50.0, 162.0, 431.0, 185.0], '\n# I. General Information\n', '1|-4'],
 [[50.0, 192.0, 541.0, 711.0],
  '1. This application is (check all that apply): New Renewal Minor Modification Major Modification a. Date of Inspection: N/A\nb. Person contacted and contact information: \nc. Site visit conducted by: d. Inspection Report Printed from BIMS attached: Yes No e. Physical Address of Site including zip code: \nf. Driving Directions if rural site and/ or no physical address: \ng. Latitude: N/A Longitude: N/A\nSource of Lat/Long & accuracy ( i.e., Google Earth, GPS, etc.):__ ________ \nII. DESCRIPTION OF INJECTION WELL(S) AND FACILITY\n1. Type of injection system: \nGeothermal

In [ ]:
import os
from convert import convert_single_pdf
def get_dir_pdfs(parent_dir):
    if not os.path.exists(parent_dir):
        raise Exception("Starting directory '%s' does not exist" % parent_dir)
    return [fn for fn in os.listdir(parent_dir) if ".pdf" in fn]

def to_refdocid(s):
    return (
        s.replace(".pdf","").replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("/", "_")
        .replace(".", "-")
        .replace("#", "")
        .replace("'", "")
    )
def rename_pdf(self, parent_dir, filename, new_filename):
    old_file = os.path.join(parent_dir, filename)
    new_file = os.path.join(parent_dir, new_filename)

    os.rename(old_file, new_file)
def run(self, parent_dir, override_existing=False):
    pdf_fns = get_dir_pdfs(parent_dir)
    for fn in pdf_fns:
        docid = to_refdocid(fn)
        self.rename_pdf(parent_dir, fn, docid + ".pdf")
        input_path = os.path.join(parent_dir, docid + ".pdf")
        output_path = os.path.join(parent_dir, docid)
        if override_existing == False and os.path.exists(output_path):
            print("skipping existing markdown directory: %s" % output_path)
            continue
        # if not os.path.exists(output_path):
        #     os.makedirs(output_path)
        # elif override_existing:
        #     shutil.rmtree(output_path)
        #     os.makedirs(output_path)

        if self.verbose:
            print("processing ", input_path)
        try:
            self.run_single(input_path, output_path)
        except Exception as e:
            print(f"Error processing {input_path}: {e}")

def run_single(self, input_path, output_path=None):
    full_text, images, out_meta, pages, text_blocks = convert_single_pdf(input_path, model_lst)
    return text_blocks

: 